In [1]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [2]:
def MIP_IM():
    attributes = ['gender', 'age', 'region', 'ethnicity']
  
    budget = 25
    m = 100
    for attribute in attributes:
        for index in range(20):
            with open('../MIP/data/networks_prob/graph_spa_500_'+str(index)+'.pickle', "rb") as f:
                main_graph = pickle.load(f)

                samples = []
                for j in range(m):
                    G = nx.DiGraph()
                    for u in main_graph.nodes():
                        G.add_node(u)
                    for u,v in main_graph.edges():
                        if main_graph[u][v]['p']> random.random():
                            G.add_edge(u, v)
                    samples.append(G)

                labels = nx.get_node_attributes(main_graph, attribute)
                label_dict = {}
                for i in range(len(main_graph.nodes())):
                    label = labels[i].encode('utf-8')
                    if label in label_dict.keys():
                        label_dict[label].append(i)
                    else:
                        label_dict[label] = [i] 
                        
                model = Model('maxmin_'+str(attribute)+'_'+str(index))
                min_value = model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS)


                mvars = []
                #active nodes
                avars = []
                #seed nodes
                svars = []
                var_seed_dict = {}
                var_active_dict = {}
                var_mean_dict = {}

                for j in range(len(main_graph.nodes())):
                    s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                    svars.append(s)
                    var_seed_dict[j] = s

                for sample_index, sample in enumerate(samples):
                    for j in range(len(main_graph.nodes())):
                        a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                        avars.append(a)
                        var_active_dict[(sample_index,j)] = a    

                mvars.append(avars)
                mvars.append(svars)

                obj_expr = quicksum(avars)
                model.setObjective(min_value, GRB.MAXIMIZE)

                model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)
                
                for sample_index, sample in enumerate(samples):
                    for i in range(len(main_graph.nodes())):
                        neighbors = nx.ancestors(sample, i) 
                        e = len(neighbors)
                        ai = var_active_dict[(sample_index,i)]
                        si = var_seed_dict[i]
                        if i in  var_mean_dict.keys():
                            var_mean_dict[i]+= ai
                        else:
                            var_mean_dict[i]= ai
                        neighbors_active_vars = []
                        neighbors_seed_vars = []
                        neighbors_active_vars.append(((e+1), ai))
                        neighbors_seed_vars.append(si)
                        for neighbor in neighbors:
                            neighbors_active_vars.append(((e+1), var_active_dict[(sample_index,neighbor)]))
                            neighbors_seed_vars.append(var_seed_dict[neighbor])
                        seed_neighbors = quicksum(neighbors_seed_vars)
                        model.addConstr(ai, GRB.LESS_EQUAL, seed_neighbors)
                        model.addConstr(seed_neighbors, GRB.LESS_EQUAL, LinExpr(neighbors_active_vars))

                mean_label_dict = {}
                for label in label_dict.keys():
                    for node in label_dict[label]:
                        if label in mean_label_dict.keys():
                            mean_label_dict[label]+= var_mean_dict[node]
                        else:
                            mean_label_dict[label]= var_mean_dict[node]
                    label_size = len(label_dict[label])
                    model.addConstr(quicksum(mean_label_dict[label]), GRB.GREATER_EQUAL, label_size *m* min_value)

                try:
                    model.optimize()
                except e:
                    print(e)

                with open('../../../../Git/influence_maximization/experiments/im500/results/fairmip/maxmin/base100/'+attribute+'/output_'+str(index)+'.txt', "w") as of:    
                    for key,value in var_seed_dict.items():
                        if(value.x > 0):
                            print(key, file=of)



'\n                with open(\'../../../../Git/influence_maximization/experiments/im500/results/fairmip/maxmin/base100/\'+attribute+\'/output_\'+str(index)+\'.txt\', "w") as of:    \n                    for key,value in var_seed_dict.items():\n                        if(value.x > 0):\n                            print(key, file=of)\n'

In [3]:
MIP_IM()

Optimize a model with 100003 rows, 50501 columns and 360512 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 59354 rows and 28819 columns
Presolve time: 1.37s
Presolved: 40649 rows, 21682 columns, 186244 nonzeros
Variable types: 1 continuous, 21681 integer (21681 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 40649 rows, 21682 columns, 186244 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   17473    7.9799091e-02   0.000000e+00   3.164217e-01      5s
Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.559473e-01, 30968 iterations, 4.46 seconds

    Nodes    |    Current Node    |     Objective Bounds     